# # setup
---

In [ ]:
%cd /data/kimgh/CenterPoint-static/det3d/ops/iou3d_nms

!python setup.py build_ext --inplace

# # Prepare data
---

In [ ]:
%cd /data/kimgh/CenterPoint

!python create_data.py nuscenes_data_prep --root_path '/data/kimgh/CenterPoint/data/nuScenes/' --version 'v1.0-mini' --nsweeps 10

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-dynamic

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-dynamic/data'
sensor ='lidar'

!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor}

In [3]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data/lidar'
sensor = 'lidar'
subsample = 10
num_process = 10

f'!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor} --subsample {subsample} --num_process {num_process}'

/data/kimgh/CenterPoint-custom/CenterPoint-static


'!python create_data.py nia_data_prep --root_path /data/kimgh/CenterPoint-custom/CenterPoint-static/data/lidar --sensor lidar --subsample 10 --num_process 10'

In [4]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data/radar'
sensor = 'radar'
subsample = 10
num_process = 10

f'!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor} --subsample {subsample} --num_process {num_process}'

/data/kimgh/CenterPoint-custom/CenterPoint-static


'!python create_data.py nia_data_prep --root_path /data/kimgh/CenterPoint-custom/CenterPoint-static/data/radar --sensor radar --subsample 10 --num_process 10'

In [7]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data/sample'
sensor = 'lidar'
subsample = 10
num_process = 10

!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor} --subsample {subsample} --num_process {num_process}

/data/kimgh/CenterPoint-custom/CenterPoint-static
no apex
2023-04-26 12:08:40.676471: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 12:08:41.645617: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No Tensorflow
exist train frames: 56 exist val frames: 56 exist test_nomal frames: 30 exist test_abnormal frames: 26
100%|██████████████████████████████████████████| 26/26 [00:00<00:00, 761.06it/s]
1
------------------------------------
/data/kimgh/CenterPoint-custom/CenterPoint-static/data/sample/infos_train_filter_True_lidar.pkl
------------------------------------
reindexing group_id: 100%|███████████████████| 11/11 [00:00<00:00, 51665.56it/s]
dataset length:  56
load 55 median_strip database infos

# # Train
---
- pm2 start train_multigpu.sh --interpreter bash --name "centerpoint_train_lidar" --no-autorestart -- 4 --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_train.py --work_dir ./result/lidar_1stsub/train --validate

- CUDA_VISIBLE_DEVICES=0,1,2,3 pm2 start train_multigpu.sh --interpreter bash --name "centerpoint_train_radar" --no-autorestart -- 4 --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_train.py --work_dir ./result/radar_allsub/train --validate --resume_from /data/kimgh/CenterPoint-custom/CenterPoint-static/result/radar_allsub/train/latest.pth

In [ ]:
%cd /data/kimgh/CenterPoint

CONFIG_PATH = '/data/kimgh/CenterPoint/configs/nusc/voxelnet/nusc_centerpoint_voxelnet_01voxel.py'
work_dir = './result/nusc_centerpoint_voxelnet_01voxel'

f'!python -m torch.distributed.launch --nproc_per_node 4 train.py {CONFIG_PATH} --work_dir {work_dir} --validate'

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal.py'
work_dir = './result/lidar/train'

!python train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_train.py'
work_dir = './result/lidar_allsub/train'

f'!python -m torch.distributed.launch --nproc_per_node 4 train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate'

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-dynamic

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-dynamic/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel.py'
work_dir = './result/lidar/train'

!python -m torch.distributed.launch --nproc_per_node 4 train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate

# # Validation
---

In [1]:
# lidar-normal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/lidar_1stsub/test/normal_epoch7'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/lidar_1stsub/train/epoch_7.pth'
sensor = 'lidar'

f'!CUDA_VISIBLE_DEVICES=3 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test' 

/data/kimgh/CenterPoint-custom/CenterPoint-static


'!CUDA_VISIBLE_DEVICES=3 python dist_test.py --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal.py --work_dir /data/kimgh/CenterPoint-custom/CenterPoint-static/result/lidar_1stsub/test/normal_epoch7 --checkpoint /data/kimgh/CenterPoint-custom/CenterPoint-static/result/lidar_1stsub/train/epoch_7.pth --sensor lidar --speed_test'

In [2]:
# lidar-abnormal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_abnormal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/lidar_1stsub/test/abnormal_epoch3'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/lidar_1stsub/train/epoch_3.pth'
sensor = 'lidar'

f'!CUDA_VISIBLE_DEVICES=3 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'

/data/kimgh/CenterPoint-custom/CenterPoint-static


'!CUDA_VISIBLE_DEVICES=3 python dist_test.py --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_abnormal.py --work_dir /data/kimgh/CenterPoint-custom/CenterPoint-static/result/lidar_1stsub/test/abnormal_epoch3 --checkpoint /data/kimgh/CenterPoint-custom/CenterPoint-static/result/lidar_1stsub/train/epoch_3.pth --sensor lidar --speed_test'

In [264]:
# radar-normal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_normal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/radar_allsub/test/normal_epoch4'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/radar_allsub/train/epoch_4.pth'
sensor = 'radar'

# !CUDA_VISIBLE_DEVICES=0 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test
f'!bash test_multigpu.sh 4 --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'

/data/kimgh/CenterPoint-custom/CenterPoint-static


'!bash test_multigpu.sh 4 --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_normal.py --work_dir /data/kimgh/CenterPoint-custom/CenterPoint-static/result/radar_allsub/test/normal_epoch4 --checkpoint /data/kimgh/CenterPoint-custom/CenterPoint-static/result/radar_allsub/train/epoch_4.pth --sensor radar --speed_test'

In [265]:
# radar-abnormal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_abnormal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/radar_allsub/test/abnormal_epoch4'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/radar_allsub/train/epoch_4.pth'
sensor = 'radar'

# !CUDA_VISIBLE_DEVICES=0 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test
f'!bash test_multigpu.sh 4 --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'

/data/kimgh/CenterPoint-custom/CenterPoint-static


'!bash test_multigpu.sh 4 --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_abnormal.py --work_dir /data/kimgh/CenterPoint-custom/CenterPoint-static/result/radar_allsub/test/abnormal_epoch4 --checkpoint /data/kimgh/CenterPoint-custom/CenterPoint-static/result/radar_allsub/train/epoch_4.pth --sensor radar --speed_test'